In [1]:
from functools import partial
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from ray import tune
from ray.tune import JupyterNotebookReporter
from ray.tune.schedulers import ASHAScheduler

from my_models import (AlexNet, VGGBlock16)

In [2]:
in_ch = 3
out_ch = 10
models_dict = {
    "AlexNet": "AlexNet({},{})".format(in_ch, out_ch),
    "VGGBlock16": "VGGBlock16({},{}, 32, 32)".format(in_ch, out_ch)
}

In [3]:
# создание папок для логов разых моделей:
checkpoint_dir="./data/checkpoints/"

for name in models_dict:    
    if not os.path.isdir(checkpoint_dir+name):
        os.makedirs(checkpoint_dir+name)


In [4]:
# исходники: https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html

In [5]:
def load_data(data_dir="./data/CIFAR"):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    trainset = torchvision.datasets.CIFAR10(
        root=data_dir, train=True, download=True, transform=transform)

    testset = torchvision.datasets.CIFAR10(
        root=data_dir, train=False, download=True, transform=transform)

    return trainset, testset

In [6]:
def train_cifar(config, model_name, epoch_num=2,
                checkpoint_dir=checkpoint_dir, data_dir=None): 
    
    net = eval(models_dict[model_name])
    
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)

    net.to(device)
    # чтобы итерировать по оптимизаторам решила ввести параметр-строку, 
    # т.к. оптимизаторы при инициализации требуют ссылку на паремтры сетки:
    optimizers_dict={
        "SGD": optim.SGD(net.parameters(), lr=config["lr"]),
        "Adam": optim.Adam(net.parameters(), lr=config["lr"])
    }
    
    criterion = config["losses"] # для итерирования разных losses
    optimizer = optimizers_dict[config["optimizers_names"]]
    """
    if checkpoint_dir: 
        model_state, optimizer_state = torch.load(
            os.path.join(checkpoint_dir+model_name, "checkpoint"))
        net.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)
    """
    trainset, testset = load_data(data_dir)

    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(
        trainset, [test_abs, len(trainset) - test_abs])

    trainloader = torch.utils.data.DataLoader(
        train_subset,
        batch_size=int(config["batch_size"]),
        shuffle=True)
    valloader = torch.utils.data.DataLoader(
        val_subset,
        batch_size=int(config["batch_size"]),
        shuffle=True)

    for epoch in range(epoch_num):  # loop over the dataset multiple times

        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        # Validation loss
        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(valloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1

        with tune.checkpoint_dir(epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save((net.state_dict(), optimizer.state_dict()), path)

        tune.report(loss=(val_loss / val_steps), accuracy=correct / total)
    print("Finished Training")

In [7]:
def main(model_name, num_samples=10, max_num_epochs=10, gpus_per_trial=2):
    data_dir = os.path.abspath("./data")
    load_data(data_dir)
    
    # заменила конфиг
    config = {        
        "lr":tune.grid_search([1e-2, 1e-1]),   
        "batch_size": tune.grid_search([100, 1000]),
        "optimizers_names":  tune.grid_search(["Adam", "SGD"]), # ключи словаря
        "losses": nn.CrossEntropyLoss() # на попробовать tune.grid_search(["""nn.MultiLabelMarginLoss(), """ nn.CrossEntropyLoss()]) 
    }
    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2)
    reporter = JupyterNotebookReporter(
        # parameter_columns=["l1", "l2", "lr", "batch_size"],
        overwrite = True,
        print_intermediate_tables = False,
        metric_columns=["loss", "accuracy", "precision"]) # немного побаловалась, не нашла что еще добавить на посмотреть
    result = tune.run(
        partial(train_cifar, data_dir=data_dir, model_name=model_name,
                checkpoint_dir=checkpoint_dir),
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=reporter,
    local_dir= checkpoint_dir+model_name)
    
    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))
    print("Best trial final validation accuracy: {}".format(
        best_trial.last_result["accuracy"]))    

    return best_trial
   

In [8]:
config = config = {        
        "lr":1e-1,   
        "batch_size": 100,
        "optimizers_names":  "Adam",
        "losses": nn.CrossEntropyLoss()
    }
model_name = "VGGBlock16"
train_cifar(config, model_name, epoch_num=2,
                checkpoint_dir=checkpoint_dir, data_dir='./data/CIFAR')

0it [00:00, ?it/s]

170500096it [00:07, 21785991.85it/s]                               


Extracting ./data/CIFAR/cifar-10-python.tar.gz to ./data/CIFAR
Files already downloaded and verified


RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/nn/parallel/parallel_apply.py", line 60, in _worker
    output = module(*input, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/torch/nn/modules/module.py", line 541, in __call__
    result = self.forward(*input, **kwargs)
  File "/notebooks/sorokina/my_models.py", line 104, in forward
    t = self.fc1(t)
  File "/usr/local/lib/python3.6/dist-packages/torch/nn/modules/module.py", line 541, in __call__
    result = self.forward(*input, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/torch/nn/modules/linear.py", line 87, in forward
    return F.linear(input, self.weight, self.bias)
  File "/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py", line 1372, in linear
    output = input.matmul(weight.t())
RuntimeError: size mismatch, m1: [1 x 832], m2: [64 x 60] at /pytorch/aten/src/THC/generic/THCTensorMathBla

In [8]:
model_name = "VGGBlock16"

best_trial = main(model_name, num_samples=1, max_num_epochs=2, gpus_per_trial=1)

2020-12-28 18:05:17,135	INFO services.py:1173 -- View the Ray dashboard at http://127.0.0.1:8265
2020-12-28 18:05:17,141	WARNING services.py:1640 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This may slow down performance! You may be able to free up space by deleting files in /dev/shm or terminating any running plasma_store_server processes. If you are inside a Docker container, you may need to pass an argument with the flag '--shm-size' to 'docker run'.
2020-12-28 18:05:19,205	WARNING experiment.py:285 -- No name detected on trainable. Using DEFAULT.
2020-12-28 18:05:19,207	INFO registry.py:65 -- Detected unknown callable for trainable. Converting to class.


KeyboardInterrupt: 

In [11]:
# файлик, который говорит, где лежит чекпоинт нужной модели:
with open("./data/best_trial_dir.txt", "w") as file:        
    file.write(best_trial.checkpoint.value+"\n")
    file.write(str(best_trial.config))